In [3]:
import sys

sys.path.append("../")

import numpy as np
import functools as ft
import networkx as nx
from graph.graph import process_dataset, transform
from hdc import hdv, bind, bundle, sbundle, ItemMemory, hdvw, hdva, cosim

In [4]:
def init_vertices(graph, d=10000):
    for n in graph:
        graph.nodes[n]["hdv"] = hdv(d)

In [5]:
def init_node_mem(graph, memory):
    for n in graph:
        xs = map(lambda x: graph.nodes[x]["hdv"], list(graph.neighbors(n)))
        graph.nodes[n]["mem"] = bundle(xs)
        memory.add_vector(f"mem{n}", graph.nodes[n]["mem"])

In [6]:
def retrain(graph, memory, threshold, iter=15):
    count = 0
    for i in range(iter):
        for n in graph:
            mem = graph.nodes[n]["mem"]
            finish = True
            for neighbor in map(
                lambda x: graph.nodes[x]["hdv"], list(nx.neighbors(graph, n))
            ):
                if cosim(mem, neighbor) < threshold:
                    mem = bundle([mem, neighbor])
                    finish = False
                    print("here")

            for non_neighbor in map(
                lambda x: graph.nodes[x]["hdv"], list(nx.non_neighbors(graph, n))
            ):
                if cosim(mem, non_neighbor) > threshold:
                    mem = bundle([mem, -non_neighbor])
                    finish = False
                    print("here")

            if finish:
                return

            graph.nodes[n]["mem"] = mem
            memory.add_vector(f"mem{n}_{i}", mem)

In [7]:
def init_graph(graph, memory):
    G = []
    for n in graph.nodes():
        G.append(bind([graph.nodes[n]["hdv"], graph.nodes[n]["mem"]]))
    G = bundle(G) / 2
    memory.add_vector("G", G)
    return G

In [8]:
def check_edge(G, A, B, threshold):
    return cosim(B, bind([G, A])) > threshold

In [9]:
def node_memory_reconstruction(G, xs, iter=15):
    if iter == 0:
        return list(map(lambda H: bind([G, H]), xs))

    mems = node_memory_reconstruction(G, xs, iter - 1)

    newMems = []
    b = [bind(x) for x in zip(mems, xs)]
    for i in range(len(xs)):
        bmem = bundle(b[:i] + b[i + 1 :])
        newMems.append(bind([xs[i], bundle([G, -bmem])]))

    return newMems

In [10]:
def graph_memory_reconstruction(G, xs, iter=15):
    raise NotImplementedError
    if iter == 0:
        return (G, zeros(len(G)))

    (Gi, Ni) = graph_memory_reconstruction(G, xs, iter - 1)
    Gii = bundle([Gi, -Ni])

    for i in range(iter):
        print(i)

    return bundle([Gi, -edges])

In [11]:
# visited = dict()
visited = {}


def shortest_path(G, A, B, xs, distance, threshold, memory):
    label, value, sim = memory.cleanup(bind([G, A]))

    if np.array_equiv(A, B):
        return 0, label

    if label in visited and visited[label][0] <= distance:
        return visited[label][1], visited[label][2]

    visited[label] = [distance, 99, "memnill"]

    neighbours = list(filter(lambda x: check_edge(G, A, x, threshold), xs))

    if len(neighbours) == 0:
        return 99, "memnill"

    dis, lab = min(
        list(
            map(
                lambda a: shortest_path(G, a, B, xs, distance + 1, threshold, memory),
                neighbours,
            )
        ),
        key=lambda x: x[0],
    )
    visited[label][1] = dis + 1
    visited[label][2] = label + lab
    return dis + 1, label + lab

---


# Tests


In [12]:
def test_check_edge(G, graph, threshold):
    count = 0

    for n in graph.nodes():
        for m in graph.nodes():
            if n == m:
                continue
            exist = graph.has_edge(n, m)
            check = check_edge(
                G, graph.nodes[n]["hdv"], graph.nodes[m]["hdv"], threshold
            )
            if exist != check:
                count += 1
                print(n, m, exist, check)

    print(count, "%.5f" % round(count / EDGES, 5))

In [13]:
def test_node_memory_reconstruction(G, graph, iter=15):
    for i in range(iter):
        memsi = node_memory_reconstruction(
            G, list(map(lambda x: graph.nodes[x]["hdv"], graph.nodes())), iter=i
        )
        print(
            f"{0}_{i:02} =>",
            "%.10f" % abs(cosim(graph.nodes[0]["mem"], memsi[0])),
        )

In [14]:
def test_shortest_path():
    mem = ItemMemory()

    g = nx.gnm_random_graph(NODES, EDGES)
    init_vertices(g, DIMENSIONS)
    init_node_mem(g, mem)
    G = init_graph(g, mem)

    for n in g.nodes():
        print(g.edges(n))

    hdvs = list(map(lambda x: g.nodes[x]["hdv"], g.nodes()))
    origin = 0

    for n in g.nodes():
        print(
            f"{n:02} =>",
            "T" if mem.cleanup(bind([G, hdvs[n]]))[0] == f"mem{n}" else "F",
            mem.cleanup(bind([G, hdvs[n]]))[0],
        )

    test_check_edge(G, g, THRESHOLD)

    for m in g.nodes():
        if origin == m:
            continue
        path = nx.shortest_path(g, origin, m)
        oracle = len(path) - 1
        visited = {}
        test = shortest_path(G, hdvs[origin], hdvs[m], hdvs, 0, THRESHOLD, mem)
        testPath = list(map(int, test[1].split("mem")[1:]))
        print(
            f"{origin}_{m:02} =>",
            oracle,
            test[0],
            "T  " if oracle == test[0] else "F X",
            path,
            testPath,
            nx.is_path(g, testPath),
        )

---


# Main


In [15]:
NODES, EDGES = 30, 150
DIMENSIONS, THRESHOLD, ITER = 10000, 0.047, 4

In [16]:
def main():
    graph = nx.gnm_random_graph(NODES, EDGES)
    memory = ItemMemory()

    init_vertices(graph, DIMENSIONS)
    init_node_mem(graph, memory)
    retrain(graph, memory, THRESHOLD, ITER)
    G = initGraph(graph, memory)
    print("G =>", G)
    # test_node_memory_reconstruction(G, graph, 1)
    # test_check_edge(G, graph, THRESHOLD)


# main()
test_shortest_path()

[(0, 23), (0, 20), (0, 1), (0, 13), (0, 7), (0, 18), (0, 25), (0, 8), (0, 6), (0, 2)]
[(1, 14), (1, 15), (1, 11), (1, 7), (1, 23), (1, 13), (1, 0), (1, 19), (1, 9), (1, 6), (1, 8)]
[(2, 22), (2, 13), (2, 28), (2, 21), (2, 10), (2, 24), (2, 14), (2, 3), (2, 5), (2, 25), (2, 9), (2, 0), (2, 19)]
[(3, 24), (3, 23), (3, 29), (3, 28), (3, 13), (3, 12), (3, 21), (3, 18), (3, 20), (3, 2), (3, 27), (3, 10), (3, 14)]
[(4, 7), (4, 26), (4, 5), (4, 19), (4, 6), (4, 22)]
[(5, 16), (5, 9), (5, 11), (5, 4), (5, 17), (5, 25), (5, 2), (5, 15)]
[(6, 15), (6, 8), (6, 22), (6, 11), (6, 26), (6, 20), (6, 4), (6, 1), (6, 28), (6, 0), (6, 12)]
[(7, 22), (7, 10), (7, 4), (7, 11), (7, 1), (7, 26), (7, 0), (7, 28), (7, 15)]
[(8, 6), (8, 26), (8, 12), (8, 14), (8, 13), (8, 11), (8, 18), (8, 29), (8, 0), (8, 1), (8, 25), (8, 19), (8, 10)]
[(9, 21), (9, 29), (9, 5), (9, 1), (9, 12), (9, 2)]
[(10, 7), (10, 2), (10, 28), (10, 26), (10, 15), (10, 17), (10, 23), (10, 11), (10, 3), (10, 8)]
[(11, 16), (11, 13), (11, 1

0_01 => 1 1 T   [0, 1] [0, 1] True
0_02 => 1 1 T   [0, 2] [0, 1] True
0_03 => 2 1 F X [0, 23, 3] [0, 1] True
0_04 => 2 1 F X [0, 7, 4] [0, 1] True
0_05 => 2 1 F X [0, 25, 5] [0, 1] True
0_06 => 1 1 T   [0, 6] [0, 1] True
0_07 => 1 1 T   [0, 7] [0, 1] True
0_08 => 1 1 T   [0, 8] [0, 1] True
0_09 => 2 1 F X [0, 1, 9] [0, 1] True
0_10 => 2 1 F X [0, 7, 10] [0, 1] True
0_11 => 2 1 F X [0, 13, 11] [0, 1] True
0_12 => 2 1 F X [0, 13, 12] [0, 1] True
0_13 => 1 1 T   [0, 13] [0, 1] True
0_14 => 2 1 F X [0, 1, 14] [0, 1] True
0_15 => 2 1 F X [0, 6, 15] [0, 1] True
0_16 => 2 1 F X [0, 23, 16] [0, 1] True
0_17 => 2 1 F X [0, 13, 17] [0, 1] True
0_18 => 1 1 T   [0, 18] [0, 1] True
0_19 => 2 1 F X [0, 1, 19] [0, 1] True
0_20 => 1 1 T   [0, 20] [0, 1] True
0_21 => 2 1 F X [0, 2, 21] [0, 1] True
0_22 => 2 1 F X [0, 6, 22] [0, 1] True
0_23 => 1 1 T   [0, 23] [0, 1] True
0_24 => 2 1 F X [0, 23, 24] [0, 1] True
0_25 => 1 1 T   [0, 25] [0, 1] True
0_26 => 2 1 F X [0, 8, 26] [0, 1] True
0_27 => 2 1 F X [0

---
